<a href="https://colab.research.google.com/github/dlkt101101/AMATH-445/blob/main/AMATH445_A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AMATH 445
## Assignment 1
Prepared by: Darren Alexander Lam Kin Teng (20977843)

# Question 1


## 1a

1. From a given confusion matrix, we will get values for true positive (TP), true negative (TN), false positive (FP) and false negative (FP).
$$Precision = \frac{TP}{TP+FP}$$
$$Recall = \frac{TP}{TP + FN}$$

**Precision** is a measure of accuracy of the positive predictions out of all the instances that the model interpreted as positive (i.e. TP + FP).\
\
**Recall** is a measure of accuracy of the positive predictions out of all actual positive cases (i.e. TP + FN).\
\
2. $$Accuracy = \frac{TP+TN}{TP+TN+FP+FN}$$
Accuracy alone can be a misleading metric for a classification problem as it does not represent the model's true performance well in the case of imbalanced data.\
For instance, if we have an imbalanced dataset where true labels are 5% of the data and 95% are negative labels. A model could predict all data points as negative. Therefore, accuracy = 95%, precision = $\frac{0}{0}$ and recall = 0%. Model performs poorly at predicting the true labels, based on the recall and precision, despite accuracy showing favourable results.

## 1b

In [84]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

Importing the dataset

In [85]:
data = pd.read_csv('/content/Cell viability and extrusion dataset V1.csv')

In [86]:
df = data.copy()
df.head()

,Reference,DOI,Final_Alginate_Conc_(%w/v),Final_Gelatin_Conc_(%w/v),Final_GelMA_Conc_(%w/v),Final_Hyaluronic_Acid_Conc_(%w/v),Final_MeHA_Conc_(%w/v),Final_NorHA_Conc_(%w/v),Final_Fibroin/Fibrinogen_Conc_(%w/v),Final_P127_Conc_(%w/v),...,Saline_Solution_Used?,EtOH_Solution_Used?,Photoinitiator_Used?,Enzymatic_Crosslinker_Used?,Matrigel_Used?,Conical_or_Straight_Nozzle,Primary/Not_Primary,Viability_at_time_of_observation_(%),Acceptable_Viability_(Yes/No),Acceptable_Pressure_(Yes/No)
0,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,96.0,Y,Y
1,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,72.0,N,N
2,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,2.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,80.0,Y,Y
3,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,0.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,96.0,Y,Y
4,Fedorovich et. al. 2008,10.1089/ten.a.2007.0158,0.0,0.0,0.0,0.0,0,0,0.0,0,...,N,N,N,N,N,S,Primary,77.0,N,N


In [87]:
df.shape

(617, 51)

In [88]:
df['Syringe_Temperature_(°C)'] = df['Syringe_Temperature_(°C)'].fillna(22)

In [89]:
null_percentage = round(df.isnull().sum()/df.shape[0] * 100,2)
null_percentage_50_names = null_percentage[null_percentage > 50].index
df.drop(labels=null_percentage_50_names, axis=1, inplace=True)
print("""Dropped columns with more than > 50% null values.
{}""".format(list(null_percentage_50_names)))


Dropped columns with more than > 50% null values. 
['Extrusion_Rate_Lengthwise_(mm/s)', 'Extrusion_Rate_Volume-wise_(mL/s)', 'Nozzle_Movement_Speed_(mm/s)', 'Fiber_Spacing_(µm)']


Imputing remaining missing values

In [110]:
remaing_missing = df.isna().sum()
remaing_missing_cols = list(remaing_missing[remaing_missing > 0].index)
print('''Here are the remaining columns with missing values:
{}'''.format(remaing_missing_cols))

Here are the remaining columns with missing values: 
['Final_Collagen_Conc_(%w/v)', 'Final_PEGMA_Conc_(%w/v)', 'CaCl2_Conc_(mM)', 'NaCl2_Conc_(mM)', 'Physical_Crosslinking_Durantion_(s)', 'Extrusion_Pressure (kPa)', 'Outer_Nozzle_Inner_Diameter_(µm)', 'Fiber_Diameter_(µm)', 'Cell_Density_(cells/mL)', 'Substrate_Temperature_(°C)', 'Conical_or_Straight_Nozzle']


In [91]:
remaing_missing_cols_object = list(df[remaing_missing_cols].dtypes[df[remaing_missing_cols].dtypes == 'object'].index)
remaing_missing_cols_object

['Fiber_Diameter_(µm)', 'Conical_or_Straight_Nozzle']

`Fiber_Diameter_(µm)` is supposed to be numerical but is type `object`.

In [105]:
df['Fiber_Diameter_(µm)'] = df['Fiber_Diameter_(µm)'].str.extract(r'(\d+\.?\d*)').astype(float)

In [ ]:
remaing_missing = df.isna().sum()
remaing_missing_cols = list(remaing_missing[remaing_missing > 0].index)
print('''Here are the remaining columns with missing values:
{}'''.format(remaing_missing_cols))

In [109]:
df.dtypes

,0
Reference,object
DOI,object
Final_Alginate_Conc_(%w/v),float64
Final_Gelatin_Conc_(%w/v),float64
Final_GelMA_Conc_(%w/v),float64
Final_Hyaluronic_Acid_Conc_(%w/v),float64
Final_MeHA_Conc_(%w/v),int64
Final_NorHA_Conc_(%w/v),int64
Final_Fibroin/Fibrinogen_Conc_(%w/v),float64
Final_P127_Conc_(%w/v),int64


In [108]:
# knnimputer = KNNImputer(n_neighbors=5)
# df[remaing_missing_cols] = knnimputer.fit_transform(df[remaing_missing_cols])

## 1c Decision Tree Classifier

## 1d Support Vector Machine (SVM)

# Question 3

# Question 4